In [4]:
from langchain_openai import OpenAIEmbeddings, AzureOpenAIEmbeddings
from langchain.vectorstores import FAISS
import os
from langchain.prompts.chat import ChatPromptTemplate
from langchain.llms.openai import OpenAI
from langchain.schema.runnable import RunnablePassthrough
from langchain.agents.agent_toolkits import SQLDatabaseToolkit
from langchain.sql_database import SQLDatabase
from langchain.agents.mrkl.base import ZeroShotAgent
from langchain.agents.mrkl.prompt import FORMAT_INSTRUCTIONS
from langchain.memory import VectorStoreRetrieverMemory
from langchain.agents.agent import AgentExecutor
from langchain.chains.llm import LLMChain

os.environ['OPENAI_API_KEY'] = "20a8e1c4c5348d62eaaa64beabbd2fa0.34ZWDCTkvM5o7PDp"
os.environ['AZURE_OPENAI_API_KEY'] = "1140131bf4f94e75b3927e82589058af"
os.environ['AZURE_OPENAI_ENDPOINT'] = "https://oliverz11ai.openai.azure.com/"


# LangSmith
# os.environ["LANGCHAIN_PROJECT"]=f"Evaluate_SQLAgent_RAG"
# os.environ["LANGCHAIN_TRACING_V2"] = "true"
# os.environ["LANGCHAIN_API_KEY"] = "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"

# 1. Prepare Documents

**Technical documents**

In [1]:
file_path = "../标签制作/voc_labels.txt"

with open(file_path, "r") as file:
    autosight_knowledge = file.read()

print(autosight_knowledge)

产品##安全性##安全架构##车身材质
产品##安全性##安全架构##车身刚性
产品##安全性##安全架构##车身结构
产品##安全性##安全架构##防撞梁/保险杠
产品##安全性##安全架构##碰撞等级
产品##安全性##安全配置##ABS防抱死
产品##安全性##安全配置##安全带
产品##安全性##安全配置##安全气囊
产品##安全性##安全配置##安全性整体评价
产品##安全性##安全配置##并线辅助
产品##安全性##安全配置##车道偏离预警系统
产品##安全性##安全配置##车身稳定控制(ESC/ESP/DSC等)
产品##安全性##安全配置##车身主动控制系统(ABC)
产品##安全性##安全配置##车外声光提醒
产品##安全性##安全配置##车载系统安全卫士
产品##安全性##安全配置##儿童安全座椅接口
产品##安全性##安全配置##辅助驾驶系统
产品##安全性##安全配置##活体遗留监测提醒
产品##安全性##安全配置##紧急救援呼叫
产品##安全性##安全配置##内置行车记录仪
产品##安全性##安全配置##疲劳驾驶提醒
产品##安全性##安全配置##牵引力控制(ASR/TCS/TRC等)
产品##安全性##安全配置##前/后排头部气囊(气帘)
产品##安全性##安全配置##入侵碰撞报警
产品##安全性##安全配置##刹车灯
产品##安全性##安全配置##胎压监测装置
产品##安全性##安全配置##弯道制动控制(CBC)
产品##安全性##安全配置##膝部气囊
产品##安全性##安全配置##制动力分配(EBD/CBC等)
产品##安全性##安全配置##主/副驾驶座安全气囊
产品##安全性##安全配置##主动刹车/主动安全系统
产品##安全性##安全配置##自适应巡航
产品##操控##操控性感受##操控性整体评价
产品##操控##操控性感受##过弯稳定性/侧倾
产品##操控##操控性感受##弯道表现
产品##操控##操控性感受##行驶稳定性
产品##操控##操控性感受##行驶指向性/循迹性
产品##操控##操控性感受##行驶中的路感
产品##操控##操控性感受##行驶中轮胎抓地感
产品##操控##操作便利性##后备厢操作便利性
产品##操控##操作便利性##中控台操作便利性
产品##操控##驾驶模式##ECO模式
产品##操控##驾驶模式##沙地

# 2. Create Vector DB

In [5]:
technical_list = [autosight_knowledge]

# embedding_function = OpenAIEmbeddings(model="embedding-2",
#                                       openai_api_base="https://open.bigmodel.cn/api/paas/v4/embeddings",
#                                       )

openai_api_key = '1140131bf4f94e75b3927e82589058af'


# 确认 AzureOpenAIEmbeddings 类的参数名称
embedding_function = AzureOpenAIEmbeddings(azure_deployment="text-embedding-3-small",
                                           openai_api_version="2023-05-15",
                                           )

# load it into FAISS
db = FAISS.from_texts(technical_list, embedding_function)

In [8]:
db.similarity_search("Calculate similarity")[0]

Document(page_content='产品##安全性##安全架构##车身材质\n产品##安全性##安全架构##车身刚性\n产品##安全性##安全架构##车身结构\n产品##安全性##安全架构##防撞梁/保险杠\n产品##安全性##安全架构##碰撞等级\n产品##安全性##安全配置##ABS防抱死\n产品##安全性##安全配置##安全带\n产品##安全性##安全配置##安全气囊\n产品##安全性##安全配置##安全性整体评价\n产品##安全性##安全配置##并线辅助\n产品##安全性##安全配置##车道偏离预警系统\n产品##安全性##安全配置##车身稳定控制(ESC/ESP/DSC等)\n产品##安全性##安全配置##车身主动控制系统(ABC)\n产品##安全性##安全配置##车外声光提醒\n产品##安全性##安全配置##车载系统安全卫士\n产品##安全性##安全配置##儿童安全座椅接口\n产品##安全性##安全配置##辅助驾驶系统\n产品##安全性##安全配置##活体遗留监测提醒\n产品##安全性##安全配置##紧急救援呼叫\n产品##安全性##安全配置##内置行车记录仪\n产品##安全性##安全配置##疲劳驾驶提醒\n产品##安全性##安全配置##牵引力控制(ASR/TCS/TRC等)\n产品##安全性##安全配置##前/后排头部气囊(气帘)\n产品##安全性##安全配置##入侵碰撞报警\n产品##安全性##安全配置##刹车灯\n产品##安全性##安全配置##胎压监测装置\n产品##安全性##安全配置##弯道制动控制(CBC)\n产品##安全性##安全配置##膝部气囊\n产品##安全性##安全配置##制动力分配(EBD/CBC等)\n产品##安全性##安全配置##主/副驾驶座安全气囊\n产品##安全性##安全配置##主动刹车/主动安全系统\n产品##安全性##安全配置##自适应巡航\n产品##操控##操控性感受##操控性整体评价\n产品##操控##操控性感受##过弯稳定性/侧倾\n产品##操控##操控性感受##弯道表现\n产品##操控##操控性感受##行驶稳定性\n产品##操控##操控性感受##行驶指向性/循迹性\n产品##操控##操控性感受##行驶中的路感\n产品##操控##操控性感受##行驶中轮胎抓地感\n产品##操控##操作便利性##后备厢操作便利

# 3. Create Tearadata Search Tool

In [17]:
from langchain.tools import BaseTool
from typing import Optional, Type

from langchain.callbacks.manager import (
    AsyncCallbackManagerForToolRun,
    CallbackManagerForToolRun,
)

In [19]:
class SQLAgentRAGRetriever(BaseTool):
    name = "sqlagent_rag_retriever"
    description = "Input to this tool is a SQL query or keyword. Output is a retrieval of related context and definitions to enhance the understanding of the query using RAG."

    def _run(
        self, query: str, run_manager: Optional[CallbackManagerForToolRun] = None
    ) -> str:
        """Use the tool to retrieve relevant documents based on the query."""
        global retriever
        relevant_docs = retriever.invoke(query)
        if len(relevant_docs) == 0 or len(query) == 0:
            return "No relevant context found for the provided query."
        else:
            return relevant_docs[0].page_content

    async def _arun(
        self, query: str, run_manager: Optional[AsyncCallbackManagerForToolRun] = None
    ) -> str:
        """Use the tool asynchronously to retrieve context. Currently not supported."""
        raise NotImplementedError("Asynchronous retrieval is not supported for this tool.")

sqlagent_rag_retriever = SQLAgentRAGRetriever()

# 4. Redefine SQL Agent

In [20]:
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage

# 创建系统消息
def create_system_message_cn():
    """
    创建系统消息，用于指导代理如何响应。
    """
    SQL_PREFIX_TEMPLATE = """
    角色：
    您是一个设计用来与SQL数据库交互的代理。您可以查询数据库并直接访问结果。

    目标：
    针对一个问题，首先进行RAG检索收集完整信息，据检索信息用户所提出的优化问题，然后创建一个SQL查询来检索所需的数据，执行此查询，并直接返回查询的结果，而不是查询本身。如果遇到错误或初始结果为空，请诊断问题，调整查询，并再次尝试。

    指示：
    - 首先进行RAG检索收集完整信息
    - 根据检索信息优化用户所提出的问题
    - 然后识别相关表格，以了解可用数据的范围。
    - 使用'sql_db_schema'工具获取这些表的结构详细信息。
    - 根据表结构信息构建一个适当的SQL查询来回答优化后的问题。
    - 使用'sql_db_query'工具执行查询。
    - 直接返回执行结果。除非用户明确请求，否则不返回SQL查询文本。
    - 如果查询执行成功但没有找到结果，请重新评估假设并相应修改查询。
    - 如果发生错误（例如，找不到表或语法错误），通过以下方式排除错误：
        - 检查表名和表结构。
        - 调整查询以修正任何错误。
        - 重新执行更正后的查询。
    - 仅关注检索与问题答案相关的列。
    - 默认情况下将结果限制为最多5个条目，除非用户指定了更多。
    - 按相关列排序结果，以首先呈现最相关的信息。
    - 不断调整和重新运行查询，直到获得有效结果或确认数据库中不存在所需数据。
    - 避免对数据库的结构或内容进行任何修改（例如，不进行INSERT、UPDATE、DELETE、DROP语句）。
    - 确保最终响应直接基于查询结果，提供简洁准确的答案。
    """
    return SystemMessage(content=SQL_PREFIX_TEMPLATE)

In [21]:
from langchain_core.messages import SystemMessage


# 创建系统消息
def create_system_message():
    SQL_PREFIX_TEMPLATE = """
    ROLE:
    You are an agent designed to interact with a SQL database. You can query the database and directly access the results.

    GOAL:
    Given a question, create a SQL query to retrieve the needed data, execute this query, and directly return the results of the query, not the query itself. If errors are encountered or the initial results are empty, diagnose the issues, adjust the query, and try again.

    INSTRUCTIONS:
    - Begin by identifying the relevant tables to understand the scope of available data.
    - Use the 'sql_db_schema' tool to obtain detailed information about the structure of these tables.
    - Construct an appropriate SQL query to answer the question based on the schema information.
    - Execute the query using the 'sql_db_query' tool.
    - Directly return the results from the execution. Do not return the SQL query text unless requested specifically by the user.
    - If the query execution is successful but no results are found, reassess the assumptions and revise the query accordingly.
    - If an error occurs (e.g., table not found or syntax error), troubleshoot the error by:
        - Reviewing the table names and schema.
        - Adjusting the query to fix any errors.
        - Re-executing the corrected query.
    - Focus only on retrieving relevant columns to answer the question.
    - Limit the results to a maximum of 5 entries by default, unless more is specified by the user.
    - Sort the results by relevant columns to present the most pertinent information first.
    - Continuously adjust and re-run the query until valid results are obtained or confirm that the data needed does not exist in the database.
    - Avoid making any modifications to the database's structure or content (e.g., no INSERT, UPDATE, DELETE, DROP statements).
    - Ensure the final response is based directly on the query results, providing concise and accurate answers.
"""
    return SystemMessage(content=SQL_PREFIX_TEMPLATE)

In [22]:
from langchain_community.utilities.sql_database import SQLDatabase
from langchain_community.agent_toolkits import SQLDatabaseToolkit
from langchain_core.prompts import ChatPromptTemplate
from langchain.agents import tool

# 初始化工具包
def initialize_toolkit(db, llm):
    """
    初始化工具包，用于与数据库交互。
    参数：
        - db: 数据库连接对象。
        - llm: 聊天模型对象。
    """
    toolkit = SQLDatabaseToolkit(db=db, llm=llm)
    tools = toolkit.get_tools()

    # Query checking
    query_check_system = """
    You are a SQL expert with a strong attention to detail.
    Double check the SQLite query for common mistakes, including:
    - Using NOT IN with NULL values
    - Using UNION when UNION ALL should have been used
    - Using BETWEEN for exclusive ranges
    - Data type mismatch in predicates
    - Properly quoting identifiers
    - Using the correct number of arguments for functions
    - Casting to the correct data type
    - Using the proper columns for joins

    If there are any of the above mistakes, rewrite the query. If there are no mistakes, just reproduce the original query.

    Execute the correct query with the appropriate tool.
    """
    query_check_prompt = ChatPromptTemplate.from_messages([("system", query_check_system),("user", "{query}")])
    query_check = query_check_prompt | llm

    @tool
    def check_query_tool(query: str) -> str:
        """
        Use this tool to double check if your query is correct before executing it.
        """
        return query_check.invoke({"query": query}).content

    # Query result checking
    query_result_check_system = """You are grading the result of a SQL query from a DB.
    - Check that the result is not empty.
    - If it is empty, instruct the system to re-try!"""
    query_result_check_prompt = ChatPromptTemplate.from_messages([("system", query_result_check_system),("user", "{query_result}")])
    query_result_check = query_result_check_prompt | llm

    @tool
    def check_result(query_result: str) -> str:
        """
        Use this tool to check the query result from the database to confirm it is not empty and is relevant.
        """
        return query_result_check.invoke({"query_result": query_result}).content

    tools.append(check_query_tool)
    tools.append(check_result)
    tools.append(sqlagent_rag_retriever)
    return tools

## Connect Database

In [23]:
from langchain_community.utilities.sql_database import SQLDatabase
from urllib.parse import quote_plus as urlquote

def connectToTheDatabase(connect_name = 'autosight_model'):
    # ClickHouse 数据库连接参数
    db_config = {
        'user': "autosight_read_only",
        'password': 'DFb7734HBdfBB33',
        'host': 'cc-2ze66w4q0tsjnd3b6.public.clickhouse.ads.aliyuncs.com',
        'port': 8123,
        'db': 'autosight_model',
        'db_type': 'clickhouse'
    }
    db_type = db_config['db_type']
    db_user = db_config['user']
    db_password = db_config['password']
    db_host = db_config['host']
    db_port = db_config['port']
    db_name = db_config['db']
    # db_name = connect_name
    db = SQLDatabase.from_uri(
        f'{db_type}://{db_user}:{urlquote(db_password)}@{db_host}:{db_port}/{db_name}?charset=utf8', view_support=True)
    return db

In [24]:
db_test = connectToTheDatabase()

In [25]:
from langchain_openai import AzureChatOpenAI, ChatOpenAI
from langgraph.prebuilt import create_react_agent
from langgraph.checkpoint.sqlite import SqliteSaver

# Step 1. Define LLM model
# chat_model_Azure = AzureChatOpenAI(
#     azure_deployment="gpt-4o",  # or your deployment
#     api_version="2023-03-15-preview",  # or your api version
# )

chat_model = ChatOpenAI(
        model_name="GLM-4",
        openai_api_base="https://open.bigmodel.cn/api/paas/v4",
        streaming=True,
        verbose=True,
    )

# Step 2. Setting connection to db and SQL Toolkits
tools = initialize_toolkit(db_test, chat_model)
system_message = create_system_message_cn()
memory = SqliteSaver.from_conn_string(":memory:")
agent_executor = create_react_agent(chat_model, tools, state_modifier=system_message, checkpointer=memory)


# 5. Test

In [26]:
question = "BBA五月份的平均库存深度是多少？"

In [27]:
config = {
"configurable": {
    # 检查点通过线程ID访问
    "thread_id": 'bob',
    }
}

for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content=question)]}, config
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'index': 0, 'id': 'call_20240808164337a06bbc798f6046d0', 'function': {'arguments': '{"query":"BBA五月份的平均库存深度"}', 'name': 'sqlagent_rag_retriever'}, 'type': 'function'}]}, response_metadata={'finish_reason': 'tool_callstool_calls', 'model_name': 'glm-4glm-4'}, id='run-5c392828-4873-4295-925b-a963b50b5789-0', tool_calls=[{'name': 'sqlagent_rag_retriever', 'args': {'query': 'BBA五月份的平均库存深度'}, 'id': 'call_20240808164337a06bbc798f6046d0', 'type': 'tool_call'}])]}}
----
{'tools': {'messages': [ToolMessage(content='\n### BBA定义\n- BBA指的是德国三大豪华汽车品牌：宝马（BMW）、奔驰（Mercedes-Benz）和奥迪（Audi）。\n- bba：常用于非正式或口语交流中，指同样的品牌组合\n- 德系三强：指的是同样的三个品牌，强调其在德国及全球市场的领先地位\n- 豪华品牌集团：可能用于描述这三个品牌在高端市场的集合\n', name='sqlagent_rag_retriever', tool_call_id='call_20240808164337a06bbc798f6046d0')]}}
----
{'agent': {'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'index': 0, 'id': 'call_20240808164340d557e81264994ca2', 'function':